In [11]:
#akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

In [12]:
#basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'

In [13]:
#ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [26]:
import pandas as pd
import numpy as np

In [15]:
#akas_df = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [16]:
#basics_df= pd.read_csv(basics_url, sep='\t', low_memory=False)

In [17]:
#ratings_df= pd.read_csv(ratings_url, sep='\t', low_memory=False)

#Processing akas

In [18]:
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


AKAs:
keep only US movies.
Replace "\N" with np.nan

In [20]:
us = akas_df['region']=='US'

In [21]:
us.sum()

1472899

In [22]:
 akas_df_final = akas_df.loc[akas_df['region']=='US']
    

In [23]:
akas_df_final.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
36,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
41,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


Handle Place Holders

In [27]:
akas_df_final = akas_df_final.replace({'\\N':np.nan})

In [28]:
akas_df_final.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


Filtering/Cleaning Steps:
Title Basics:
Replace "\N" with np.nan
Eliminate movies that are null for runtimeMinutes
Eliminate movies that are null for genre
keep only titleType==Movie
keep startYear 2000-2022
Eliminate movies that include "Documentary" in genre (see tip below)
Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [29]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [30]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10253861 entries, 0 to 10253860
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 704.1+ MB


In [31]:
basics_df_final = basics_df.replace({'\\N':np.nan})
basics_df_final.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [33]:
#Eliminate movies that are null for runtimeMinutes

basics_df_final['runtimeMinutes'].isnull().sum()

7161875

In [34]:
basics_df_final['runtimeMinutes'].notnull().sum()

3091986

In [58]:
filtered_df=basics_df_final.loc[basics_df_final['runtimeMinutes'].notnull()]
filtered_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
10253811,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,0,2013,NaN,49,Documentary
10253817,tt9916766,tvEpisode,Episode #10.15,Episode #10.15,0,2019,NaN,43,"Family,Game-Show,Reality-TV"
10253852,tt9916840,tvEpisode,Horrid Henry's Comic Caper,Horrid Henry's Comic Caper,0,2014,NaN,11,"Adventure,Animation,Comedy"
10253859,tt9916856,short,The Wind,The Wind,0,2015,NaN,27,Short


In [59]:
filtered_df.shape

(3091986, 9)

In [60]:
#Eliminate movies that are null for genre
filtered_df=filtered_df.loc[filtered_df['genres'].notnull()]
filtered_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
10253811,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,0,2013,NaN,49,Documentary
10253817,tt9916766,tvEpisode,Episode #10.15,Episode #10.15,0,2019,NaN,43,"Family,Game-Show,Reality-TV"
10253852,tt9916840,tvEpisode,Horrid Henry's Comic Caper,Horrid Henry's Comic Caper,0,2014,NaN,11,"Adventure,Animation,Comedy"
10253859,tt9916856,short,The Wind,The Wind,0,2015,NaN,27,Short


keep only titleType==Movie
keep startYear 2000-2022

In [61]:
filtered_df['titleType'].value_counts()

tvEpisode       1585244
short            617398
movie            390938
video            185768
tvMovie           92921
tvSeries          92471
tvSpecial         19299
tvMiniSeries      18095
tvShort            8752
videoGame           339
Name: titleType, dtype: int64

In [62]:
#keep only titleType==Movie
filtered_df=filtered_df.loc[filtered_df['titleType']=='movie']

In [63]:
filtered_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
...,...,...,...,...,...,...,...,...,...
10253711,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama
10253752,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
10253779,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,NaN,100,Documentary
10253801,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN,116,Drama


In [64]:
filtered_df = filtered_df.dropna(subset=['startYear'])
filtered_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
...,...,...,...,...,...,...,...,...,...
10253711,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama
10253752,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
10253779,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,NaN,100,Documentary
10253801,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN,116,Drama


In [68]:
filtered_df = filtered_df.copy()

In [73]:
filtered_df['startYear'] = filtered_df['startYear'].astype(int)
filtered_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
...,...,...,...,...,...,...,...,...,...
10253711,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama
10253752,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
10253779,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,NaN,100,Documentary
10253801,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN,116,Drama


In [74]:
#keep startYear 2000-2022
filtered_df.loc[(filtered_df['startYear']>=2000 ) & (filtered_df['startYear']<=2022)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13081,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,94,Documentary
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61111,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67485,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
67663,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
...,...,...,...,...,...,...,...,...,...
10253711,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama
10253752,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
10253779,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,NaN,100,Documentary
10253801,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN,116,Drama


Eliminate movies that include "Documentary" in genre (see tip below)


In [80]:
# Exclude movies that are included in the documentary category.
is_documentary = filtered_df['genres'].str.contains('documentary',case=False)
basics_df_final = filtered_df[~is_documentary]


In [81]:
#
basics_df_final

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
930,tt0000941,movie,Locura de amor,Locura de amor,0,1909,NaN,45,Drama
...,...,...,...,...,...,...,...,...,...
10253543,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"
10253582,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,NaN,84,Thriller
10253627,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"
10253711,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama


In [ ]:
#Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [82]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics_df_final['tconst'].isin(akas_df_final['titleId'])
keepers

8            True
570          True
587          True
672          True
930         False
            ...  
10253543     True
10253582    False
10253627     True
10253711    False
10253801    False
Name: tconst, Length: 291026, dtype: bool

In [83]:
basics_df_final = basics_df_final[keepers]
basics_df_final


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
1273,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"
...,...,...,...,...,...,...,...,...,...
10253000,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
10253394,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019,NaN,97,"Comedy,Drama,Fantasy"
10253534,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
10253543,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"



Ratings:
Replace "\N" with np.nan (if any)
Keep only US movies (Use AKAs table)

In [85]:
ratings_df_final= ratings_df.replace({'\\N':np.nan})

In [86]:
ratings_df_final.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2003
1,tt0000002,5.8,269
2,tt0000003,6.5,1896
3,tt0000004,5.5,178
4,tt0000005,6.2,2681


In [87]:
#Keep only US
keepers =ratings_df_final['tconst'].isin(akas_df_final['titleId'])
keepers

0           True
1           True
2          False
3          False
4           True
           ...  
1361672    False
1361673    False
1361674    False
1361675    False
1361676    False
Name: tconst, Length: 1361677, dtype: bool

In [88]:
ratings_df_final = ratings_df_final[keepers]
ratings_df_final

,tconst,averageRating,numVotes
0,tt0000001,5.7,2003
1,tt0000002,5.8,269
4,tt0000005,6.2,2681
5,tt0000006,5.0,183
6,tt0000007,5.4,839
...,...,...,...
1361639,tt9916200,8.1,238
1361640,tt9916204,8.2,274
1361647,tt9916348,8.3,18
1361648,tt9916362,6.4,5575


Data Downloading

In [89]:
#  making new folder with os
import os
os.makedirs('Project3Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Project3Data/")

[]

In [ ]:
#Saving Compressed .csv.gz Files

In [91]:
## Save current dataframe to file.
basics_df_final.to_csv("Project3Data/title_basics.csv.gz",compression='gzip',index=False)

In [92]:
## Save current dataframe to file.
ratings_df_final.to_csv("Project3Data/title_ratings.csv.gz",compression='gzip',index=False)

In [93]:
## Save current dataframe to file.
akas_df_final.to_csv("Project3Data/title_akas.csv.gz",compression='gzip',index=False)

In [94]:
# Open saved file and preview again
basics = pd.read_csv("Project3Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
3,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
4,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"


In [96]:
# Open saved file and preview again
ratings = pd.read_csv("Project3Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2003
1,tt0000002,5.8,269
2,tt0000005,6.2,2681
3,tt0000006,5.0,183
4,tt0000007,5.4,839


In [97]:
# Open saved file and preview again
akas = pd.read_csv("Project3Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0
